In [2]:
import numpy as np

from ase import Atoms
from ase.io import read, write

from pyace import create_multispecies_basis_config
from pyace.activelearning import compute_B_projections

In [9]:
basis_config = {
  "deltaSplineBins": 0.001,
  "elements": ['Au'],

  "embeddings": {
    "ALL": {
      "npot": 'FinnisSinclairShiftedScaled', # ?
      "fs_parameters": [ 1, 1],
      "ndensity": 1, # ?
    },
  },

  "bonds": {
    "ALL": {
      "radbase": "SBessel",
      "radparameters": [ 5.25 ],
      "rcut": 6, # ?
      "dcut": 0.01, # ?
    }
  },

  "functions": {
    # "number_of_functions_per_element": 1000,
    "ALL": {
        "nradmax_by_orders": [ 8,8,6],
        "lmax_by_orders"   : [ 0,6,4] }
  }
}

In [10]:
basis = create_multispecies_basis_config(basis_config)
basis

BBasisConfiguration(deltaSplineBins=0.001, funcspecs_blocks=['Au', ])

### Compare FCC structures with 1% strain and unstrained

In [11]:
from ase.build import bulk, make_supercell
import numpy as np

a1 = 3.58
supercell = 1
fcc1 = bulk('Au', 'fcc', a=a1, cubic=True)

fcc2 = fcc1.copy()
fcc2.set_cell(0.99 * fcc2.cell, scale_atoms=True)

# fcc1 = make_supercell(fcc1, np.eye(3) * supercell)
# fcc2 = make_supercell(fcc2, np.eye(3) * supercell)

x1 = compute_B_projections(basis, [fcc1])[0]
x2 = compute_B_projections(basis, [fcc2])[0]

distance = np.linalg.norm(x1 - x2)
print(distance)

95.45151481769105
